In [6]:
import os
import groq

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [7]:
#pip install pydantic

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.schema.output_parser import StrOutputParser

In [9]:
prompt=ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatGroq()
output_parser = StrOutputParser()

chain = prompt | model | output_parser #pipe
chain.invoke({"topic": "bears"})

'Why do bears hate shoes?\n\nThey like to run around in their bear feet!'

In [10]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Boss is angry", "Sarah left the job"],
    embedding=CohereEmbeddings()
)
retriever=vectorstore.as_retriever()

In [11]:
retriever.get_relevant_documents("Who is angry?")

[Document(page_content='Boss is angry'),
 Document(page_content='Sarah left the job')]

In [12]:
retriever.get_relevant_documents("Who left the job?")

[Document(page_content='Sarah left the job'),
 Document(page_content='Boss is angry')]

In [13]:
template = """Answer the question based only on the following context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain.schema.runnable import RunnableMap
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser


In [15]:
chain.invoke({"question": "who is angry?"})

'Based on the information provided, the "boss" is angry. The first document states "Boss is angry". There is no information in the provided context about Sarah\'s feelings or the reason for her leaving the job.'

In [16]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [17]:
inputs.invoke({"question": "who left the work?"})

{'context': [Document(page_content='Sarah left the job'),
  Document(page_content='Boss is angry')],
 'question': 'who left the work?'}

In [40]:
tools = [
    {"type": "function",
     "function": {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
            "airport_code": {
                "type": "string",
                "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
            },
        },
    },
    {"type": "function",
     "function": {
         "name": "sports_search",
         "description": "Search for news of recent sport events",
         "parameters": {
             "type":"object",
             "properties": {
                "team_name":{
                    "type": "string",
                    "description": "The sports team to search for"
                },
             },
         },
         "required":["team_name"]
        }
    }
  ]

In [41]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model=ChatGroq(temperature=0).bind(tools=tools)

In [43]:
runnable=prompt | model
runnable.invoke("input: How did the patriots do yesterday?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yww2', 'function': {'arguments': '{"team_name":"Patriots"}', 'name': 'sports_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.147831131, 'completion_tokens': 93, 'prompt_time': 0.546019904, 'prompt_tokens': 1339, 'queue_time': None, 'total_time': 0.693851035, 'total_tokens': 1432}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-66f906be-03e8-4802-97ed-241456bb5d29-0', tool_calls=[{'name': 'sports_search', 'args': {'team_name': 'Patriots'}, 'id': 'call_yww2'}])

In [46]:
import json
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author and first line"

model=ChatGroq(temperature=0.0)
chain = model | StrOutputParser() | json.loads
chain.invoke(challenge)

JSONDecodeError: Invalid \escape: line 6 column 7 (char 78)